In [1]:
import numpy as np

from IPython.display import display, Math

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as subplots

import plotly.io as pio

# I2C

## DAC Parameters

In [2]:
VCC = 5

V_bus = VCC

V_OL = 0.3 * VCC
I_OL = 0.01

C_b = 10e-12
t_r = 20e-9 + 0.1 * C_b

display(Math("V_{bus} = " + f"{V_bus}V"), Math("V_{OL} =" + f"{V_OL}V~" + "~I_{OL}=" + f"{I_OL}A"), Math("C_b=" + f"{C_b*1e12}pF~" + "t_r=" + f"{t_r*1e9}ns"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Min Resistance

In [3]:
RP_min = (V_bus - V_OL)/I_OL

display(Math("R_{p~(min)}=" + f"{RP_min}\Omega"))

<IPython.core.display.Math object>

## Max Resistance

In [4]:
RP_max = t_r / (0.8473 * C_b)

display(Math("R_{p~(max)}=" + f"{RP_max}\Omega"))

<IPython.core.display.Math object>

### Multiple DACs

In [5]:
n = 2

RP_max_mult = t_r / (0.8473 * n * C_b)
RP_max_compare = 1/(n/RP_max)

display(Math("R_{p~(max)}~\mathrm{with~n~DACs}=" + f"{RP_max_mult}\Omega"),
        Math("\mathrm{n~parallel~} R_{p~(max)}=" + f"{RP_max_compare}\Omega"))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Boost Converter

## Parameters

In [6]:
Uin = 5 #V
Ua = 24 #V
Uref = 1.255 #V

R2 = 10e3 #Ohm

f = 1e3 #Switching Frequency

## Feedback (R1/C7)

$U_a = \cfrac{R_1 + R_2}{R_2} \cdot U_{ref}$

$f_{sw} = \cfrac{1}{2 \pi R_1 C_7}$

In [7]:
R1 = (Ua/Uref - 1) * R2
C7 = 1/(2 * np.pi * f * R1)

display(Math("R_{1}~=" + f"{R1*1e-3}k\Omega"),
        Math("C_{7}=" + f"{C7*1e12}pF"))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Amplifier

## Parameters

In [8]:
V = 4
Uin = 4
R3 = 15e3

## Amplification

In [9]:
R4 = R3/(V - 1)

display(Math("R_{4}~=" + f"{R4*1e-3}k\Omega"), 
        Math("R_{3}~=" + f"{R3*1e-3}k\Omega"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## AC Coupling

In [10]:
fg = 4000

### Input

$\cfrac{U_a}{U_{in}} = \cfrac{sC_8\cdot \frac{1}{2}R_{8/9}}{1 + sC_8\cdot \frac{1}{2}R_{8/9}}$

$f_g = \cfrac{1}{2 \pi C_8 \cdot \frac{1}{2}R_{8/9}}$

In [11]:
#R8 = R9 = 10e3
#C8 = 1/(np.pi * fg * R8)


R8 = R9 = 100e3
C8 = 1/(np.pi * fg * 1/2 * R8)

display(Math("R_{8/9}~=" + f"{R8*1e-3}k\Omega"),
        Math("C_{8}~=" + f"{C8*1e9}nF"))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Feedback

$A = \cfrac{1 + sC_3(R_3 + R_4)}{1 + sC_3R_4}$

$fg = \cfrac{1}{2 \pi C_3R_4}$

In [12]:
C9 = 1/(2 * np.pi * R4 * fg)

display(Math("C_{10}~=" + f"{C9*1e9}nF"))

<IPython.core.display.Math object>

In [13]:
C9 = 10e-9
fg_real = 1/(2 * np.pi * R4 * C9)

display(Math("f_{g}~=" + f"{fg_real}Hz"))

<IPython.core.display.Math object>

In [14]:
fg2 = 1/(2 * np.pi * C9 * (R4 + R3))

display(Math("f_{g2}~=" + f"{fg2}Hz"))

<IPython.core.display.Math object>

### Output

$A = \cfrac{sC_{10} \cdot Z_{10}}{1 + sC_{10} \cdot Z_{10}}$

$f_g = \cfrac{1}{2 \pi C_{10} \cdot Z_{10}}$

$Z_{10} = R_{10} || Z_M$

In [15]:
CM = 45.6e-9
ESR = 16


### Stability

In [16]:
A0 = 10**(120/20)
GBW = 5.25e6

wg = 2*np.pi*GBW/A0

#### Without load

$ A_d = \cfrac{A_0}{1 + \cfrac{s}{w_g}}$

$ k = \cfrac{1 + s C_9 R_4}{1 + s C_9 (R_4 + R_3)}$

$ g = A_d \cdot k$

In [17]:
s = lambda w: np.multiply(w, 1j)
Ad = lambda w: A0/(1 + s(w) / wg)
k = lambda w: (1 + C9 * R4 * s(w))/(1 + C9 * (R4 + R3) * s(w))

g = lambda w: Ad(w) * k(w)

In [23]:
w = np.logspace(1, 7, 100)
f = w/(2*np.pi)

G = g(w)

fig = subplots.make_subplots(rows=2, cols=1)
fig.update_layout(
    xaxis_title=r"$f \text{ in } Hz$",
    yaxis_title=r"$\text{Magnitude in } dB$",
    xaxis2_title=r"$f \text{ in } Hz$",
    yaxis2_title=r"$\text{Phase in } ^\circ$",
    xaxis=dict(type="log", range=[1, 6]),
    xaxis2=dict(type="log", range=[1, 6]),
    yaxis=dict(range=[-10, 125]),
    yaxis2=dict(range=[-180, 10]),
    showlegend=False,
)
fig.add_scatter(x=f, y=20 * np.log10(np.abs(G)), name="magnitude", row=1, col=1)
fig.add_scatter(x=f, y=180/np.pi * np.angle(G), name="phase", row=2, col=1)

#fig.show(renderer="browser")
pio.write_image(fig, "amp_stability.pdf", width=800, height=550, engine="kaleido")
fig.show()